In [1]:
import pandas as pd;
from  mlxtend.frequent_patterns import association_rules, apriori

# Read Dataset

In [2]:
sales_receipt = pd.read_csv("dataset/201904 sales reciepts.csv")
sales_receipt.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [3]:
sales_receipt['transaction_id'].value_counts().reset_index()

,transaction_id,count
0,97,67
1,49,66
2,45,65
3,274,65
4,252,64
...,...,...
4198,4123,1
4199,4122,1
4200,4114,1
4201,4097,1


In [4]:
product = pd.read_csv('dataset/product.csv')
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


# Data Wrangling

converting raw data into a more uasable format for analysis

# Merge Data

In [5]:
# pulled required columns from the original data
sales_receipt = sales_receipt[["transaction_id", 'transaction_date', 'customer_id', 'sales_outlet_id', 'product_id', 'quantity']]
product = product[["product_id", "product_category", "product"]]

#merging the data (joining the both the dtasets on product_id)

dataset = pd.merge(sales_receipt, product, on="product_id", how = "left")
dataset.head()

,transaction_id,transaction_date,customer_id,sales_outlet_id,product_id,quantity,product_category,product
0,7,2019-04-01,558,3,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,781,3,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,788,3,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,683,3,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,99,3,34,1,Coffee,Jamaican Coffee River Sm


# Remove Sizes

In [6]:
dataset[dataset["product"].str.contains("Dark chocolate")]['product'].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [7]:
dataset["product"].nunique()

80

In [8]:
dataset["product"] = dataset["product"].str.replace(" Rg", "")
dataset["product"] = dataset["product"].str.replace(" Sm", "")
dataset["product"] = dataset["product"].str.replace(" Lg", "")

In [9]:
dataset["product"].nunique()

45

In [10]:
dataset["product"].unique()

array(['Traditional Blend Chai', 'Brazilian', 'Serenity Green Tea',
       'Our Old Time Diner Blend', 'Jamaican Coffee River', 'Ethiopia',
       'English Breakfast', 'Sustainably Grown Organic', 'Earl Grey',
       'Cappuccino', 'Espresso shot', 'Latte', 'Dark chocolate',
       'Columbian Medium Roast', 'Oatmeal Scone', 'Morning Sunrise Chai',
       'Peppermint', 'Jumbo Savory Scone', 'Lemon Grass',
       'Chocolate Chip Biscotti', 'Spicy Eye Opener Chai',
       'Ginger Biscotti', 'Chocolate Croissant', 'Hazelnut Biscotti',
       'Cranberry Scone', 'Scottish Cream Scone ', 'Croissant',
       'Almond Croissant', 'Ginger Scone', 'Ouro Brasileiro shot',
       'Organic Decaf Blend', 'Chocolate syrup', 'Hazelnut syrup',
       'Carmel syrup', 'Sugar Free Vanilla syrup',
       'Jamacian Coffee River', 'Guatemalan Sustainably Grown',
       'Civet Cat', 'Chili Mayan', 'Primo Espresso Roast',
       'Brazilian - Organic', 'I Need My Bean! Diner mug',
       'Espresso Roast', 'I Need 

# remove unnecessary products


In [11]:

products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [12]:
dataset = dataset[dataset['product']. isin(products_to_take)]
dataset.head()

,transaction_id,transaction_date,customer_id,sales_outlet_id,product_id,quantity,product_category,product
16,108,2019-04-01,65,3,40,1,Coffee,Cappuccino
17,112,2019-04-01,90,3,37,2,Coffee,Espresso shot
20,127,2019-04-01,116,3,41,2,Coffee,Cappuccino
21,134,2019-04-01,189,3,38,2,Coffee,Latte
22,135,2019-04-01,131,3,40,1,Coffee,Cappuccino


In [13]:
dataset[['product', 'product_category']].drop_duplicates()

,product,product_category
16,Cappuccino,Coffee
17,Espresso shot,Coffee
21,Latte,Coffee
26,Dark chocolate,Drinking Chocolate
29,Oatmeal Scone,Bakery
35,Jumbo Savory Scone,Bakery
65,Chocolate Chip Biscotti,Bakery
96,Ginger Biscotti,Bakery
106,Chocolate Croissant,Bakery
134,Hazelnut Biscotti,Bakery


## clean transactions

There is chance that the recommendation system gets confused when there are multiple transactions getting to single customer id. So we are concating transactionid with customerid to make it unique.

In [14]:
dataset['transaction'] = dataset['transaction_id'].astype(str) + "_" + dataset['customer_id'].astype(str)
dataset.head()

,transaction_id,transaction_date,customer_id,sales_outlet_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,65,3,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,90,3,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,116,3,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,189,3,38,2,Coffee,Latte,134_189
22,135,2019-04-01,131,3,40,1,Coffee,Cappuccino,135_131


There are people who purchase one item and leave

In [15]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
num_of_items_for_each_transaction

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24
...,...,...
8381,135_523,1
8382,130_157,1
8383,121_465,1
8384,118_748,1


In [16]:
dataset[dataset['transaction'] == '206_0']

,transaction_id,transaction_date,customer_id,sales_outlet_id,product_id,quantity,product_category,product,transaction
10955,206,2019-04-07,0,8,71,1,Bakery,Chocolate Croissant,206_0
10956,206,2019-04-07,0,8,71,1,Bakery,Chocolate Croissant,206_0
11652,206,2019-04-08,0,3,71,1,Bakery,Chocolate Croissant,206_0
12258,206,2019-04-08,0,5,71,1,Bakery,Chocolate Croissant,206_0
12259,206,2019-04-08,0,5,71,1,Bakery,Chocolate Croissant,206_0
12847,206,2019-04-08,0,8,71,1,Bakery,Chocolate Croissant,206_0
12848,206,2019-04-08,0,8,71,1,Bakery,Chocolate Croissant,206_0
13485,206,2019-04-09,0,3,71,1,Bakery,Chocolate Croissant,206_0
13486,206,2019-04-09,0,3,71,1,Bakery,Chocolate Croissant,206_0
14553,206,2019-04-09,0,8,71,1,Bakery,Chocolate Croissant,206_0


In [17]:
valid_transactions = num_of_items_for_each_transaction[num_of_items_for_each_transaction['count']>1]['transaction'].tolist()
valid_transactions[:10]

['209_0',
 '206_0',
 '204_0',
 '208_0',
 '203_0',
 '207_0',
 '205_0',
 '151_0',
 '323_0',
 '301_0']

In [18]:
dataset = dataset[dataset['transaction'].isin(valid_transactions)]
dataset

,transaction_id,transaction_date,customer_id,sales_outlet_id,product_id,quantity,product_category,product,transaction
34,199,2019-04-01,112,3,41,2,Coffee,Cappuccino,199_112
35,199,2019-04-01,112,3,79,1,Bakery,Jumbo Savory Scone,199_112
54,296,2019-04-01,328,3,39,1,Coffee,Latte,296_328
55,296,2019-04-01,328,3,79,1,Bakery,Jumbo Savory Scone,296_328
64,357,2019-04-01,530,3,41,2,Coffee,Cappuccino,357_530
...,...,...,...,...,...,...,...,...,...
49880,736,2019-04-29,0,8,38,1,Coffee,Latte,736_0
49883,742,2019-04-29,0,8,41,1,Coffee,Cappuccino,742_0
49884,742,2019-04-29,0,8,74,1,Bakery,Ginger Biscotti,742_0
49885,746,2019-04-29,0,8,37,2,Coffee,Espresso shot,746_0


In [19]:
dataset.shape

(10189, 9)

# Product Trends

In [20]:
dataset['product_category'].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

In [21]:
dataset['product'].value_counts()

product
Cappuccino                  1290
Latte                       1256
Dark chocolate               969
Chocolate Croissant          636
Espresso shot                628
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Ginger Scone                 417
Jumbo Savory Scone           357
Croissant                    355
Chocolate Chip Biscotti      352
Cranberry Scone              350
Almond Croissant             347
Hazelnut Biscotti            338
Oatmeal Scone                334
Ginger Biscotti              314
Name: count, dtype: int64

# Popularity Recommendation Engine

In [22]:
product_recommendations = dataset.groupby(['product', "product_category"]).count().reset_index()
product_recommendations.head()

,product,product_category,transaction_id,transaction_date,customer_id,sales_outlet_id,product_id,quantity,transaction
0,Almond Croissant,Bakery,347,347,347,347,347,347,347
1,Cappuccino,Coffee,1290,1290,1290,1290,1290,1290,1290
2,Carmel syrup,Flavours,561,561,561,561,561,561,561
3,Chocolate Chip Biscotti,Bakery,352,352,352,352,352,352,352
4,Chocolate Croissant,Bakery,636,636,636,636,636,636,636


In [23]:
product_recommendations = product_recommendations[['product', 'product_category', 'transaction_id']]
product_recommendations = product_recommendations.rename(columns={'transaction_id' : "number of transactions"})

In [24]:
product_recommendations.head()

,product,product_category,number of transactions
0,Almond Croissant,Bakery,347
1,Cappuccino,Coffee,1290
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,352
4,Chocolate Croissant,Bakery,636


In [25]:
product_recommendations.to_csv('api/recommendation_objects/popularity_recommencation.csv')

# Apriori Recommendation Engine

In [26]:
train_basket = (dataset.groupby(['transaction', 'product']).count().reset_index())

train_basket

,transaction,product,transaction_id,transaction_date,customer_id,sales_outlet_id,product_id,quantity,product_category
0,1000_0,Dark chocolate,1,1,1,1,1,1,1
1,1000_0,Oatmeal Scone,1,1,1,1,1,1,1
2,1001_8306,Cappuccino,1,1,1,1,1,1,1
3,1001_8306,Carmel syrup,1,1,1,1,1,1,1
4,1002_0,Carmel syrup,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
8391,9_0,Croissant,1,1,1,1,1,1,1
8392,9_0,Dark chocolate,3,3,3,3,3,3,3
8393,9_0,Ginger Scone,2,2,2,2,2,2,2
8394,9_0,Latte,3,3,3,3,3,3,3


In [27]:
train_basket = train_basket[['transaction', 'product', 'transaction_id']]
train_basket = train_basket.rename(columns={"transaction_id": "count"})
train_basket.head()

,transaction,product,count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1


Other way of doing above thing is 

In [28]:
train_basket = (dataset.groupby(['transaction', 'product'])['product'].count().reset_index(name="count"))
train_basket.head()

,transaction,product,count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1


widen the datafrmae. Above one is a long format

In [29]:
my_basket = train_basket.pivot_table(index='transaction', columns='product', values = "count")
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1001_8306,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002_0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1004_5383,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1005_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN


In [30]:
my_basket = my_basket.fillna(0)
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0


loop over data frame and convert to 0's and 1's

In [31]:
def encode_units(x):
    if x <= 0:
        return 0
    if x > 0:
        return 1

my_basket= my_basket.map(encode_units)
my_basket.head(10)


product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1005_5559,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1006_0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1007_8375,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1008_0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [32]:
frequent_items = apriori(my_basket, min_support=0.05, use_colnames=True)
frequent_items

/Volumes/D/AI/CoffeeShopApp/.venv/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115646,(Almond Croissant)
1,0.388889,(Cappuccino)
2,0.191232,(Carmel syrup)
3,0.112623,(Chocolate Chip Biscotti)
4,0.135676,(Chocolate Croissant)
5,0.188964,(Chocolate syrup)
6,0.116024,(Cranberry Scone)
7,0.114135,(Croissant)
8,0.277022,(Dark chocolate)
9,0.209373,(Espresso shot)


In [89]:
rules_basket = association_rules(df=frequent_items, num_itemsets = 1, metric="lift", min_threshold=1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,1.0,0.008314,1.133333,0.176432,0.118090,0.117647,0.298905
1,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,1.0,0.008314,1.024775,0.255319,0.118090,0.024176,0.298905
2,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,1.0,0.025409,1.115717,0.611791,0.171364,0.103715,0.352050
3,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,1.0,0.025409,1.436567,0.500152,0.171364,0.303896,0.352050
4,(Almond Croissant),(Latte),0.115646,0.382086,0.054422,0.470588,1.231629,1.0,0.010235,1.167171,0.212660,0.122762,0.143227,0.306511


In [90]:
rules_basket[rules_basket['antecedents'] == {"Latte"}].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
71,(Latte),(Sugar Free Vanilla syrup),0.382086,0.200302,0.108844,0.284866,1.422182,1.0,0.032311,1.118249,0.480415,0.229848,0.105745,0.414131
33,(Latte),(Carmel syrup),0.382086,0.191232,0.107710,0.281899,1.474121,1.0,0.034643,1.126260,0.520509,0.231331,0.112105,0.422570
45,(Latte),(Chocolate syrup),0.382086,0.188964,0.103175,0.270030,1.428997,1.0,0.030974,1.111053,0.485842,0.220517,0.099953,0.408015
69,(Latte),(Hazelnut syrup),0.382086,0.171580,0.101285,0.265084,1.544961,1.0,0.035727,1.127231,0.570848,0.223893,0.112871,0.427696
53,(Latte),(Croissant),0.382086,0.114135,0.057067,0.149357,1.308605,1.0,0.013458,1.041407,0.381651,0.129948,0.039761,0.324679
38,(Latte),(Chocolate Croissant),0.382086,0.135676,0.055178,0.144411,1.064381,1.0,0.003338,1.010209,0.097889,0.119281,0.010106,0.275548
67,(Latte),(Ginger Scone),0.382086,0.133409,0.055178,0.144411,1.082472,1.0,0.004204,1.012860,0.123300,0.119869,0.012696,0.279005
5,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,1.0,0.010235,1.031236,0.304358,0.122762,0.030290,0.306511
65,(Latte),(Ginger Biscotti),0.382086,0.106198,0.054044,0.141444,1.331890,1.0,0.013467,1.041053,0.403272,0.124456,0.039434,0.325170
49,(Latte),(Cranberry Scone),0.382086,0.116024,0.051398,0.134520,1.159416,1.0,0.007067,1.021371,0.222518,0.115059,0.020924,0.288759


# Save in Json

In [91]:
product_categories = dataset[['product', 'product_category']].drop_duplicates().set_index('product').to_dict()['product_category']
product_categories

{'Cappuccino': 'Coffee',
 'Jumbo Savory Scone': 'Bakery',
 'Latte': 'Coffee',
 'Chocolate Chip Biscotti': 'Bakery',
 'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Chocolate Croissant': 'Bakery',
 'Dark chocolate': 'Packaged Chocolate',
 'Cranberry Scone': 'Bakery',
 'Croissant': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours'}

In [92]:
recommendations_json = {}
antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents'] == antecedent].sort_values('confidence', ascending=False)

    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _, row in df_rec.iterrows():
        rec_object = row['consequents']
        for rec_object in rec_object:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists=True
            if already_exists:
                continue
            rec = {
                "product": rec_object,
                "product_category": product_categories[rec_object],
                'confidence': row['confidence']
            }

            recommendations_json[key].append(rec)
            

In [37]:
import pprint
pprint.pp(recommendations_json)

{'Almond Croissant': [{'product': 'Dark chocolate',
                       'product_category': 'Packaged Chocolate',
                       'confidence': 0.4967320261437908},
                      {'product': 'Latte',
                       'product_category': 'Coffee',
                       'confidence': 0.47058823529411764},
                      {'product': 'Cappuccino',
                       'product_category': 'Coffee',
                       'confidence': 0.46078431372549017}],
 'Cappuccino': [{'product': 'Sugar Free Vanilla syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.29057337220602525},
                {'product': 'Chocolate syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.2818270165208941},
                {'product': 'Carmel syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.26433430515063167},
                {'product': 'Hazelnut syrup',
           

In [38]:
import json
with open("api/recommendation_objects/apriori_recommendations.json", 'w') as json_file:
    json.dump(recommendations_json, json_file)